In [ ]:
from ssd import build_ssd
from layers.box_utils import *
import os
import sys
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse
import torchvision
import pickle
from layers import box_utils
from layers import Detect
from layers import functions
from layers import modules
import torch.nn.functional as F
from math import sqrt as sqrt
from itertools import product as product

from torch.autograd import Function
from layers.box_utils import decode, nms

In [ ]:
# 詳細模型結構在 ssd.py
ssd_net = build_ssd("train", size=300, num_classes=21)
ssd_net.load_weights("./demo/ssd300_mAP_77.43_v2.pth")

In [ ]:
cfg = {
    'num_classes': 21,
    'lr_steps': (80000, 100000, 120000),
    'max_iter': 120000,
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'min_dim': 300,
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'variance': [0.1, 0.2],
    'clip': True,
    'name': 'VOC'
}

aspect_ratios：使用 6 張 Feature Map，每一張上方有預設的 anchor boxes，而 Boxes aspect ratio 可以自己設定

feature_maps：使用 feature map 大小為 [38x38, 19x19, 10x10, 5x5, 3x3, 1x1]

min_sizes 和 max_sizes 可藉由下面的算式算出，由作者自行設計

steps：Feature map 回放回原本 300*300 的比例，如 38 要回放為 300 大概就是 8 倍

variance：Training 的一個 trick，加速收斂，詳見：https://github.com/rykov8/ssd_keras/issues/53

min_sizes 和 max_sizes 的計算

In [14]:
# source: https://blog.csdn.net/gbyy42299/article/details/81235891
import math

min_dim = 300

# 產生 Default Box 的 layer，可以更改。許多改進都是基於此處
# conv4_3 => 38 x 38
# fc7 => 19 x 19
# conv6_2 => 10 x 10
# conv7_2 => 5 x 5
# conv8_2 => 3 x 3
# conv9_2 => 1 x 1
mbox_source_layers = ['conv4_3', 'fc7', 'conv6_2', 'conv7_2', 'conv8_2', 'conv9_2']

# 長寬比
# 這裡即是論文中所說的 Smin = 0.2 和 Smax = 0.9 的初始值，經過下面的運算即可得到 min_sizes 和 max_sizes
min_ratio = 20
max_ratio = 90

# 取長寬的間距步長（這裡等於 17），可以用一個具體的數值代替
step = int(math.floor((max_ratio - min_ratio) / (len(mbox_source_layers) - 2)))

min_sizes = []
max_sizes = []
for ratio in range(min_ratio, max_ratio + 1, step):
    # 從 min_ratio 至 max_ratio，每隔 17 就取一個值賦值給 ratio
    min_sizes.append(min_dim * ratio / 100.)
    max_sizes.append(min_dim * (ratio + step) / 100.)

min_sizes = [min_dim * 10 / 100.] + min_sizes
max_sizes = [min_dim * 20 / 100.] + max_sizes

# steps 需要仔細理解，即計算卷積層產生的 Default Box 距離原圖的步長，先驗框中心點的坐標會乘以 step
# 相當於從特徵映射位置映射回原圖位置，比如 conv4_3 輸出特徵圖大小為 38 * 38，而輸入的圖片為 300 * 300
# 所以 38 * 8 約等於 300，所以映射步長為 8
steps = [8, 16, 32, 64, 100, 300]
aspect_ratios = [
    [2],
    [2, 3],
    [2, 3],
    [2, 3],
    [2],
    [2]
]
 
print('min_sizes: ', min_sizes)
print('max_sizes: ', max_sizes)

min_sizes:  [30.0, 60.0, 111.0, 162.0, 213.0, 264.0]
max_sizes:  [60.0, 111.0, 162.0, 213.0, 264.0, 315.0]


Default anchor boxes 數量：

38 × 38 × 4 + 19 × 19 × 6 + 10 × 10 × 6 + 5 × 5 × 6 + 3 × 3 × 4 + 1 × 1 × 4 = 8732

In [ ]:
class PriorBox(object):
    """Compute priorbox coordinates in center-offset form for each source
    feature map.
    """
    def __init__(self, cfg):
        super(PriorBox, self).__init__()
        self.image_size = cfg['min_dim']
        
        # number of priors for feature map location (either 4 or 6)
        self.num_priors = len(cfg['aspect_ratios'])
        self.variance = cfg['variance'] or [0.1]
        self.feature_maps = cfg['feature_maps']
        self.min_sizes = cfg['min_sizes']
        self.max_sizes = cfg['max_sizes']
        self.steps = cfg['steps']
        self.aspect_ratios = cfg['aspect_ratios']
        self.clip = cfg['clip']
        self.version = cfg['name']
        
        for v in self.variance:
            if v <= 0:
                raise ValueError('Variances must be greater than 0')

    def forward(self):
        mean = []
        
        # 依照 Feature map 大小找出所有的 pixel 中心
        # 下方的兩個 loop 會找出 W 個 x 軸 pixel 對上 W 個 y 軸 pixel，假如現在是在 38 x 38 的 feature map 上，就會有 38 x 38 個值
        # ex. [0,1],[0,2]..[0,37] [1,1],[1,2]..[1,37]..........[37,37]
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):
                f_k = self.image_size / self.steps[k] ## 如self.steps==8，就是先將原圖size normalize(/300)後再乘上8
                # unit center x,y
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # aspect_ratio: 1
                # rel size: min_size
                # normalization
                # 小的正方形 box
                
                s_k = self.min_sizes[k] / self.image_size
                mean += [cx, cy, s_k, s_k]

                # aspect_ratio: 1
                # rel size: sqrt(s_k * s_(k+1))
                # 大的正方形box
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]

                # rest of aspect ratios
                for ar in self.aspect_ratios[k]:
                    '''aspect ratio 2,3'''
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    '''aspect ratio 1/2,1/3'''
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]
        
        # back to torch land
        output = torch.Tensor(mean).view(-1, 4)
        if self.clip:
            output.clamp_(max=1, min=0)
        return output

In [ ]:
priorBox_demo = PriorBox(cfg)

print(priorBox_demo.forward().shape)

Loss 如何設計

In [ ]:
class MultiBoxLoss(nn.Module):

    def __init__(self, num_classes, overlap_thresh, prior_for_matching,
                 bkg_label, neg_mining, neg_pos, neg_overlap, encode_target,
                 use_gpu=True):
        super(MultiBoxLoss, self).__init__()
        self.use_gpu = use_gpu
        # 預測類別
        self.num_classes = num_classes
        # 判定為正樣本的 threshold，一般設為0.5
        self.threshold = overlap_thresh
        # background 自己會有一類，不用 Label，假如我們有 20 類一樣標註 0-19，下方會自己空出一類給 background
        self.background_label = bkg_label
        self.encode_target = encode_target
        self.use_prior_for_matching = prior_for_matching
        # OHEM，找出分得最不好的樣品，也就是 confidence score 比較低的正負樣品
        self.do_neg_mining = neg_mining
        # 負樣品與正樣品的比例，通常是3:1
        self.negpos_ratio = neg_pos
        self.neg_overlap = neg_overlap
        self.variance = cfg['variance']
     

    def forward(self, predictions, targets):
        # 預測會輸出三個值
        # 1. loc shape: bounding box 資訊
        #       torch.size(batch_size, num_priors, 4)
        # 2. conf shape: 每一個 bounding box 的信心程度
        #       torch.size(batch_size, num_priors, num_classes)
        # 3. priors shape: 預設的 Default box
        #       torch.size(num_priors, 4)
        loc_data, conf_data, priors = predictions
        num = loc_data.size(0)
        priors = priors[:loc_data.size(1), :]
        num_priors = (priors.size(0))
        num_classes = self.num_classes

        # match priors (default boxes) and ground truth boxes
        loc_t = torch.Tensor(num, num_priors, 4)
        conf_t = torch.LongTensor(num, num_priors)
        for idx in range(num):
            truths = targets[idx][:, :-1].data
            labels = targets[idx][:, -1].data
            defaults = priors.data
            # jaccard 計算每一個 bounding box 與 ground truth 的 IOU
            match(self.threshold, truths, defaults, self.variance, labels,
                  loc_t, conf_t, idx)
        if self.use_gpu:
            loc_t = loc_t.cuda()
            conf_t = conf_t.cuda()
        
        # 用 Variable 包裝
        loc_t = Variable(loc_t, requires_grad=False)
        conf_t = Variable(conf_t, requires_grad=False)

        pos = conf_t > 0
        num_pos = pos.sum(dim=1, keepdim=True)


        pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
        loc_p = loc_data[pos_idx].view(-1, 4)
        loc_t = loc_t[pos_idx].view(-1, 4)
        # smooth L1 loss（計算 bounding box regression）
        loss_l = F.smooth_l1_loss(loc_p, loc_t, size_average=False)

        # Compute max conf across batch for hard negative mining
        batch_conf = conf_data.view(-1, self.num_classes)
        loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1, 1))

        # Hard Negative Mining
        loss_c = loss_c.view(num, -1)
        loss_c[pos] = 0
        # 排列 confidence 的分數
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)
        num_pos = pos.long().sum(1, keepdim=True)
        # 負樣品取出數量 == negpos_ratio * num_pos
        num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)
        neg = idx_rank < num_neg.expand_as(idx_rank)

        # Confidence Loss Including Positive and Negative Examples
        pos_idx = pos.unsqueeze(2).expand_as(conf_data)
        neg_idx = neg.unsqueeze(2).expand_as(conf_data)
        conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, self.num_classes)
        targets_weighted = conf_t[(pos+neg).gt(0)]
        # 用 cross entropy 做分類
        loss_c = F.cross_entropy(conf_p, targets_weighted, size_average=False)

        # Sum of losses: L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N
        # double 轉成 torch.float64
        N = num_pos.data.sum().double()
        loss_l = loss_l.double()
        loss_c = loss_c.double()
        loss_l /= N
        loss_c /= N
        return loss_l, loss_c


## 產生我們Loss function，注意這裡的class要包含背景

In [ ]:
use_cuda = False
criterion = MultiBoxLoss(21, 0.5, True, 0, False, 3, 0.5,False, use_cuda)

## 基本設定

In [ ]:
net = build_ssd('train', size=300, num_classes=21)
use_pretrained = True

if use_pretrained:
    net.load_weights('./demo/ssd300_mAP_77.43_v2.pth')

In [ ]:
# 要不要使用gpu
use_cuda = False

# tensor type 會因 cpu 或 gpu 而有所不同
if torch.cuda.is_available():
    if args.cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    
    if not args.cuda:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

# 使用 gpu 時可以開啟 DataParallel，但當Input是不定大小時，需要關掉
if Use_cuda:
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True

#　使用 gpu 時，模型要轉成 cuda 的模式
if Use_cuda:
    net = net.cuda()
    
batch_size = 1
learning_rate = 0.00001 / batch_size
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

訓練

In [ ]:
# 輸入影像格式，假設 batch size 為 4
image_in = torch.tensor(torch.rand(4, 3, 300, 300), dtype=torch.float32)

# Label 格式，沒有固定長度，看圖像中有幾個 label 就有幾個
label_0 = [
       [ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000]
]

label_1 = [
       [ 0.1804,  0.6076,  0.7701,  0.8485, 13.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 11.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 7.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 5.0000]
]

label_2 = [
       [ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 14.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000]
]

label_3 = [[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000]
]

In [ ]:
epochs = 300
iteration = 1000

for epoch in range(epochs):
    n = 0
    loss_sum = []
    loc_loss = []
    conf_loss = []

    for number__ in range(iteration) :
        # 需要先使用 Variable 來包裝 tensor 才能輸入到模型
        if use_cuda:
            image = Variable(image_in.cuda())
            y = [
                Variable(torch.tensor(label_0).cuda(), volatile=True),
                Variable(torch.tensor(label_1).cuda(), volatile=True),
                Variable(torch.tensor(label_2).cuda(), volatile=True),
                Variable(torch.tensor(label_3).cuda(), volatile=True)
            ]
        else:
            image = Variable(image_in)
            y = [
                Variable(torch.tensor(label_0), volatile=True),
                Variable(torch.tensor(label_1), volatile=True),
                Variable(torch.tensor(label_2), volatile=True),
                Variable(torch.tensor(label_3), volatile=True)
            ]

        # Forward Pass
        out = net(image)

        # Regression Loss and Classification Loss
        loss_l, loss_c = criterion(out, y)
        loss = loss_l + loss_c

        # Backward
        loss.backward()

        loc_loss.append(loss_l.data.cpu().numpy())
        conf_loss.append(loss_c.data.cpu().numpy())
        loss_sum.append(loss.data.cpu().numpy())
        
        # 更新參數
        optimizer.step()
        
        # 清空 Gradients
        optimizer.zero_grad()
        
        n += 1
        if n % 10 == 0:
            print('Bounding Box Regression Loss: ', np.mean(loc_loss))
            print('Classification Loss: ', np.mean(conf_loss))
    
    # store weights
    torch.save(ssd_net.state_dict(),'weights/weights.pth')